# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,14.45,64,94,5.81,NZ,1723350444
1,1,port-aux-francais,-49.3500,70.2167,4.24,92,100,13.71,TF,1723350445
2,2,beitbridge,-22.2167,30.0000,17.10,27,0,1.11,ZW,1723350446
3,3,west island,-12.1568,96.8225,28.99,65,67,4.12,CC,1723350447
4,4,carnarvon,-24.8667,113.6333,22.68,63,0,4.30,AU,1723350449


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,carnarvon,-24.8667,113.6333,22.68,63,0,4.30,AU,1723350449
81,81,bethel,41.3712,-73.4140,21.42,81,0,1.54,US,1723350542
89,89,macapa,0.0389,-51.0664,26.99,89,0,3.09,BR,1723350552
98,98,mahdia,35.5047,11.0622,24.97,70,0,0.63,TN,1723350565
140,140,maintirano,-18.0667,44.0167,23.56,73,0,2.85,MG,1723350620
197,197,fort bragg,35.1390,-79.0060,25.91,91,0,1.54,US,1723350690
230,230,'ain el turk,35.7438,-0.7693,26.38,89,0,2.06,DZ,1723350731
289,289,ghadamis,30.1337,9.5007,26.79,29,0,4.10,LY,1723350807
300,300,berezivka,47.2043,30.9078,23.52,53,0,4.23,UA,1723350822
318,318,calatayud,41.3535,-1.6432,26.24,29,0,2.13,ES,1723350843


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,carnarvon,AU,-24.8667,113.6333,63,
81,bethel,US,41.3712,-73.4140,81,
89,macapa,BR,0.0389,-51.0664,89,
98,mahdia,TN,35.5047,11.0622,70,
140,maintirano,MG,-18.0667,44.0167,73,
197,fort bragg,US,35.1390,-79.0060,91,
230,'ain el turk,DZ,35.7438,-0.7693,89,
289,ghadamis,LY,30.1337,9.5007,29,
300,berezivka,UA,47.2043,30.9078,53,
318,calatayud,ES,41.3535,-1.6432,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
macapa - nearest hotel: Equatorial Hotel
mahdia - nearest hotel: La Maison Bleue
maintirano - nearest hotel: Hotel Safari
fort bragg - nearest hotel: Airborne Inn Lodging
'ain el turk - nearest hotel: Hôtel Chems
ghadamis - nearest hotel: فندق عين الفارس
berezivka - nearest hotel: Готель Березівка
calatayud - nearest hotel: Arco San Miguel
tamanrasset - nearest hotel: فندق الأمان
medicine hat - nearest hotel: Ramada
longview - nearest hotel: Texas Lodge
elko - nearest hotel: No hotel found
roswell - nearest hotel: Best Western Plus Hotel
linxia chengguanzhen - nearest hotel: 如家酒店
balqash - nearest hotel: No hotel found
xining - nearest hotel: Qinghai hotel
portland - nearest hotel: the Hoxton
toliara - nearest hotel: Ambary
la palma - nearest hotel: La Quinta Inn & Suites Buena Park
eregli - nearest hotel: Nupelda Caddesi
bilohirsk - nearest hotel: Гузель
greenville - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
4,carnarvon,AU,-24.8667,113.6333,63,No hotel found
81,bethel,US,41.3712,-73.4140,81,Hampton Inn Danbury
89,macapa,BR,0.0389,-51.0664,89,Equatorial Hotel
98,mahdia,TN,35.5047,11.0622,70,La Maison Bleue
140,maintirano,MG,-18.0667,44.0167,73,Hotel Safari
197,fort bragg,US,35.1390,-79.0060,91,Airborne Inn Lodging
230,'ain el turk,DZ,35.7438,-0.7693,89,Hôtel Chems
289,ghadamis,LY,30.1337,9.5007,29,فندق عين الفارس
300,berezivka,UA,47.2043,30.9078,53,Готель Березівка
318,calatayud,ES,41.3535,-1.6432,29,Arco San Miguel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)